In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def relu(x): return (x>0) * x

In [ ]:
def relu_derivative(x): return x > 0

In [ ]:
alpha, iterations, hidden_size = (0.005, 300, 100)
pixels_per_image, num_labels = (784, 10)

In [ ]:
from torchvision.datasets import MNIST
train_dataset = MNIST(root='./data', train=True, download=True)
test_dataset = MNIST(root='./data', train=False, download=True)
import torch.nn.functional as F
x_train, y_train, x_test, y_test = train_dataset.data / 255., F.one_hot(train_dataset.targets, num_classes=10), test_dataset.data / 255., F.one_hot(test_dataset.targets, num_classes=10)
x_train = x_train.reshape(-1, 28 ** 2)
x_test = x_test.reshape(-1, 28 ** 2)
images, labels = x_train.numpy(), y_train.numpy()
test_images, test_labels = x_test.numpy(), y_test.numpy()

In [ ]:
images = images[:1000]
labels = labels[:1000]

In [ ]:
np.random.seed(1)
weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1
for j in range(iterations):
    error, correct_cnt = 0, 0
    for i in range(images.shape[0]):
        layer_0 = images[i:i+1]
        layer_1 = relu(layer_0 @ weights_0_1)
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = layer_1 @ weights_1_2
        error += np.sum((labels[i:i+1] - layer_2) ** 2)
        correct_cnt += int(np.argmax(layer_2) ==  np.argmax(labels[i:i+1]))
        layer_2_delta = layer_2 - labels[i:i+1]
        layer_1_delta = layer_2_delta @ weights_1_2.T * relu_derivative(layer_1)
        layer_1_delta *= dropout_mask
        weights_1_2 -= layer_1.T @ layer_2_delta * alpha
        weights_0_1 -= layer_0.T @ layer_1_delta * alpha
    if(j%10 == 0):
        test_error = 0.0
        test_correct_cnt = 0
        for i in range(len(test_images)):
            layer_0 = test_images[i:i+1]
            layer_1 = relu(np.dot(layer_0,weights_0_1))
            layer_2 = np.dot(layer_1, weights_1_2)
            test_error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
            test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))
        print("\n" + \
        "I:" + str(j) + \
        " Test-Err:" + str(test_error/ float(len(test_images)))[0:5] +\
        " Test-Acc:" + str(test_correct_cnt/ float(len(test_images)))+\
        " Train-Err:" + str(error/ float(len(images)))[0:5] +\
        " Train-Acc:" + str(correct_cnt/ float(len(images))))